In [ ]:
import os, sys
#from docxtpl import DocxTemplate
#import docx2txt
import docx
from docx_replace import docx_replace

#import inflect
import pandas as pd
import json
import re
from enum import Enum


# Import indicator data for calculating values

In [ ]:
# location where indicator data is stored
data_path = "../indicator data/"

indicatorMeta = pd.read_csv(data_path+"indicatorMeta.csv")
indicatorData = pd.read_csv(data_path+"indicatorData.csv").sort_index(axis=1)


# Opening SIDS JSON file
with open(data_path+'undpSids.json', 'r') as openfile:
  
    # Reading from json file
    undpSids = json.load(openfile)

# Create a list of SIDS iso-3 codes
SIDS = [] 
for i in undpSids.keys():
    SIDS.append(undpSids[i])
    
# create a dictionary with SIDS iso-3 codes as keys and country names as values
sidsNames = {}
sidsNames = dict([(value, key) for key, value in undpSids.items()])

In [ ]:
# Opening SIDS JSON file
with open(data_path+'region.json', 'r') as openfile:
  
    # Reading from json file
    undpSidsRegion = json.load(openfile)

# Create a list of SIDS iso-3 codes

In [ ]:
# Create regional lists
pacificRegion = []
carribeanRegion = []
aisRegion = []
for i in undpSidsRegion:
    if i["region"] == "Caribbean":
        carribeanRegion.append(i["iso"])
    elif i["region"] == "AIS":
        aisRegion.append(i["iso"])
    elif i["region"] == "Pacific":
        pacificRegion.append(i["iso"])
    

# Define the year for which you want to update values of analysis to

In [ ]:
year = 2016

# Define Indicators, statical measurement names and other codes that will be used in document

In [ ]:
# statistics that will be calculated
class stats(Enum):
    minSids = 'SIDS minimum'
    minGlobal = 'Global minimum'
    maxSids = 'SIDS maximum'
    maxGlobal = 'Global maximum'
    aveSids = 'SIDS average'
    aveGlobal = 'Global average'
    minPacific = 'pacific minimum'
    maxPacific = 'pacific maximum'
    avePacific = 'pacific average'
    minCarribean = 'Carribean minimum'
    maxCarribean = 'Carribean maximum'
    aveCarribean = 'Carribean average'
    minAIS = 'AIS minimum'
    maxAIS = 'AIS maximum'
    aveAIS = 'AIS average'
# values that will be inserted into the text
class outputs(Enum):
    countryName = 'name of country'
    value = 'statistical value calculated'
    year = 'year under consideration'

# import input files document

In [ ]:
input_path = "../word doc/input/"

output_path = "../word doc/output/"

In [ ]:
files = ["EPI Analysis demo input.docx", "EPI Analysis demo input FONT.docx"]

In [ ]:
doc = docx.Document(input_path +  files[0])
fontDoc = docx.Document(input_path + files[1])


# Define statistic calculating function

In [ ]:
def statCalculator(code_list):

    assert len(code_list) in [1,3], "code length is wrong"
    
    # Return year
    if code_list[-1] == outputs.year.name:
        return str(year)
    else:
        requiredStat = code_list[1]
        requiredOutput = code_list[2]
        indicator = code_list[0]
        # Calculate stat
        if (requiredStat == stats.minSids.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(SIDS))][[str(year),"Country Code"]].min()
        elif (requiredStat == stats.minGlobal.name):
            data = indicatorData[indicatorData["Indicator Code"]==indicator][[str(year),"Country Code"]].min()
        elif (requiredStat == stats.maxSids.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(SIDS))][[str(year),"Country Code"]].max()
        elif (requiredStat == stats.maxGlobal.name):
            data = indicatorData[indicatorData["Indicator Code"]==indicator][[str(year),"Country Code"]].max()
        elif (requiredStat == stats.aveSids.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(SIDS))][[str(year),"Country Code"]].mean()
        elif (requiredStat == stats.aveGlobal.name):
            data = indicatorData[indicatorData["Indicator Code"]==indicator][[str(year),"Country Code"]].mean()
        elif (requiredStat == stats.minPacific.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(pacificRegion))][[str(year),"Country Code"]].min()
        elif (requiredStat == stats.maxPacific.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(pacificRegion))][[str(year),"Country Code"]].max()
        elif (requiredStat == stats.avePacific.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(pacificRegion))][[str(year),"Country Code"]].mean()
        elif (requiredStat == stats.minCarribean.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(carribeanRegion))][[str(year),"Country Code"]].min()
        elif (requiredStat == stats.maxCarribean.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(carribeanRegion))][[str(year),"Country Code"]].max()
        elif (requiredStat == stats.aveCarribean.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(carribeanRegion))][[str(year),"Country Code"]].mean()
        elif (requiredStat == stats.minAIS.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(aisRegion))][[str(year),"Country Code"]].min()
        elif (requiredStat == stats.maxAIS.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(aisRegion))][[str(year),"Country Code"]].max()
        elif (requiredStat == stats.aveAIS.name):
            data = indicatorData[(indicatorData["Indicator Code"]==indicator)&(indicatorData["Country Code"].isin(aisRegion))][[str(year),"Country Code"]].mean()

        # Return required output
        if requiredOutput == outputs.value.name:
            return str(round(data[str(year)],2))
        elif requiredOutput == outputs.countryName.name:
            return sidsNames[data["Country Code"]]
def processor(code):
    code_list = code.strip("{}").split(",")
    return statCalculator(code_list=code_list)
    

# Replace  with font and style loss

### Replace coded texts in paragraphs (those between curly brackets according to the enumeration classes)

In [ ]:
for p in doc.paragraphs:
    temp = re.findall(r'\{.*?\}', p.text)
    if len(temp) > 0:
        result = [processor(i) for i in temp]
        for ind,val in enumerate(temp):
            p.text = p.text.replace(val, result[ind],1)
    print(p.text)


### Replace coded texts in tables (those between curly brackets according to the enumeration classes)

In [ ]:
for table in doc.tables:
    for row in table.rows:
        for cell in row.cells:
            for p in cell.paragraphs:
 
                temp = re.findall(r'\{.*?\}', p.text)
                if len(temp) > 0:
                    result = [processor(i) for i in temp]
                    for ind,val in enumerate(temp):
                        p.text = p.text.replace(val, result[ind],1)
                print(p.text)

In [ ]:
os.getcwd()

In [ ]:
 for roots,dirs,files in os.walk(os.getcwd()):
        print(roots)

In [ ]:
# save new word document with replaced values
doc.save(output_path + "EPI Analysis GENERATED.docx")

# Replace without font and style loss [NOT WORKING]

REFERENCE: https://github.com/ivanbicalho/python-docx-replace

In [ ]:
temp = []
for p in fontDoc.paragraphs:
    print(p.text)
    temp.extend(re.findall(r'\{.*?\}', p.text))

for table in fontDoc.tables:
    for row in table.rows:
        for cell in row.cells:
            for p in cell.paragraphs:
                temp.extend(re.findall(r'\{.*?\}', p.text))

result = [processor(i) for i in temp]
temp = ["$"+code for code in temp]


In [ ]:
for ind,val in enumerate(temp):
    docx_replace(fontDoc, val=result[ind])

In [ ]:
fontDoc.save(output_path + "EPI Analysis FONT GENERATED.docx")

In [ ]:
#code = '{epi-EPI,aveSids,value}'